In [ ]:
%run ../visualisation.ipynb
%run ../read_graph.ipynb

In [ ]:
from igraph import *
import random

In [ ]:
def getPermutation(graph):
    permut = []
    for x in range(graph.vcount()):
        permut.append(x)
    random.shuffle(permut)
    random.shuffle(permut)
    return permut

In [ ]:
def GetWeightSum(g):
    sum = 0
    for x in g.es:
        sum += x["weight"]
    return sum

def GetWeightSumInCommunity(g,communityNum):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== communityNum)
    for x in vertices:
        for y in vertices:
            sum+=g[x,y]
    return sum

def GetWeightOfLinksToKInCommunity(g,k,communityNum):
    sum = 0
    vertices = g.vs.select(lambda v: v["community"]== communityNum)
    for v in vertices:
        if k!=v.index:
            sum += g[k,v]
    return sum

def GetWeightOfLinksToCommunity(g,communityNum):
    sum = 0
    param = 0.75 # interesting stuff
    vertices = g.vs.select(lambda v: v["community"]== communityNum)
    for v in vertices:
        sum += GetWeightedDegree(g,v)
    return sum * param

def GetWeightedDegree(g,k):
    sum = 0
    for y in g.neighbors(k):
        sum += g[k,y] 
    return sum

In [ ]:
def GetModularityChange2(g,i,community):
    change = 0
    value1 = (GetWeightSumInCommunity(g,community) + GetWeightOfLinksToKInCommunity(g,i,community)*2)/(2*WeightSum(g)) - (( (GetWeightOfLinksToCommunity(g,i,community) + GetWeightedDegree(g,i))/(2*GetWeightSum(g)))**2)
    value2 = (GetWeightSumInCommunity(g,community))/(2*GetWeightSum(g)) - (GetWeightOfLinksToCommunity(g,i,community)/(2*GetWeightSum(g)))**2 - (GetWeightedDegree(g,i)/(2*GetWeightSum(g)))**2
    return value1-value2

def GetModularityChange(g,i,community):
    change=0
    m = GetWeightSum(g)
    value1= GetWeightOfLinksToKInCommunity(g,i,community)/m - (GetWeightOfLinksToCommunity(g,community)*GetWeightedDegree(g,i))/(2*(m**2))
    return value1


In [ ]:
def ChangeCommunity(comtable,comfrom,comto):
    i=0
    for x in comtable:
        if x == comfrom:
            comtable[i] = comto
        i += 1
def GetIndices(comtable,com):
    l = []
    i = 0
    for x in comtable:
        if x == com:
            l.append(i)
        i += 1
    return l

In [ ]:
def GetVerticesInCommunity(g,com):
    vertices = g.vs.select(lambda v: v["community"]== com)
    return [v.index for v in vertices]

def GetWeightOfLinksBetweenCommunities(g,com1,com2):
    vert1 = GetVerticesInCommunity(g,com1)
    vert2 = GetVerticesInCommunity(g,com2)
    ret = 0
    if(com1 != com2):
        for v1 in vert1:
            for v2 in vert2:
                ret += g[v1,v2]
    else:
        ret = GetWeightSumInCommunity(g,com1)
    return ret

In [ ]:
def MergeCommunities(g):
    newgraph = Graph()
    for com1 in set(g.vs["community"]):
        newgraph.add_vertex(community =com1)
    i=0
    communities = newgraph.vs["community"]
    for com1 in communities:
        for com2 in communities[i:]:
            weight=GetWeightOfLinksBetweenCommunities(g,com1,com2)
            if weight>0:
                es = newgraph.add_edge([v["community"] for v in newgraph.vs].index(com1),[v["community"] for v in newgraph.vs].index(com2))
                es["weight"]=weight
        i+=1
        
    return newgraph

In [ ]:
def LouvainIteration(g,permut,comtable):
    changedtable = comtable.copy()
    waschange = False
    for x in range(g.vcount()):
        prevcom = g.vs[permut[x]]["community"]
        maxmodchange = 0.0
        maxcommunity = -1
        for v in g.neighbors(permut[x]):
            g.vs[permut[x]]["community"] = g.vs[v]["community"]

            modchange = GetModularityChange(g,permut[x],g.vs[v]["community"])

            if(maxmodchange < modchange):
                maxmodchange = modchange
                maxcommunity = g.vs[v]["community"]
           
        g.vs[permut[x]]["community"] = prevcom

        if maxcommunity != -1 and maxcommunity != prevcom:
            waschange=True
            g.vs[permut[x]]["community"] = maxcommunity
            for i in GetIndices(comtable,prevcom):
                changedtable[i] = maxcommunity
    comtable[:] = changedtable[:]
    return waschange

In [ ]:
def LouvainAlogrithm(g):
    graph=g.copy()
    for x in range(graph.vcount()):
        graph.vs[x]["community"]=x
    comtable = graph.vs["community"]
    permut = getPermutation(graph)
    flag=True
    while flag:
        flag = LouvainIteration(graph,permut,comtable)
        graph = MergeCommunities(graph)
        permut = getPermutation(graph)

    return comtable

In [ ]:
def GetModularity(g):
    m = GetWeightSum(g)
    sum = 0
    for i in range(g.vcount()):
        for j in range(g.vcount()):
            if(g.vs[i]["community"] == g.vs[j]["community"]):
                sum += (g[i,j]-((GetWeightedDegree(g,i)*GetWeightedDegree(g,j))/(2*m)))
    mod = sum /(2*m)
    return mod

In [ ]:
g = Graph()
g.add_vertices(10)
g.add_edges([(0,4), (0,3),(0,2),(0,1),(1,3),(1,2),(1,5),(2,3),(2,4),(3,5),(4,5),(4,6),(4,8),(5,7),(5,9),(6,7),(6,8),(6,9),(7,8),(7,9),(8,9)])
g.es["weight"] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

comtable=LouvainAlogrithm(g)

for v in g.vs:
    v["communities"] = []
    v["communities"].append(comtable[v.index])
plot_full(g, "full.svg")

In [ ]:
g = Graph()
g.add_vertices(16)
g.add_edges([(0,2), (0,3),(0,5),(1,2),(1,4),(1,7),(2,4),(2,5),(3,7),(4,10),(5,7),(5,11),(6,7),(6,11),(8,9),(8,11),(8,15),(8,14),(9,14),(9,12),(10,11),(10,12),(10,13),(10,14),(11,13)])
g.es["weight"] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

comtable=LouvainAlogrithm(g)

for v in g.vs:
    v["communities"] = []
    v["communities"].append(comtable[v.index])
plot_full(g, "full.svg")

In [ ]:
g = Graph()
g.add_vertices(10)
g.add_edges([(0,1), (1,2),(2,3),(3,4),(4,5),(5,6),(6,7),(7,8),(8,9)])
g.es["weight"] = [1,1,1,1,1,1,1,1,1,1]

comtable=LouvainAlogrithm(g)

for v in g.vs:
    v["communities"] = []
    v["communities"].append(comtable[v.index])
plot_full(g, "full.svg")

In [ ]:
g=Graph()
g.add_vertices(39)
g.add_edges([(0,1),(3,4),(6,7),(8,9),(9,10),(10,11),(10,12),(11,12),(10,13),(13,14),(14,15),(15,16),(14,17),(14,18),(14,19),(19,20),(20,21),(21,22),(22,23),(22,24),(23,24),(24,25),(25,26),(24,26),(24,27),(27,28),(28,29),(29,18),(28,30),(30,31),(31,32),(31,22),(31,33),(33,34),(34,35)])
g.es["weight"] = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

comtable=LouvainAlogrithm(g)

for v in g.vs:
    v["communities"] = []
    v["communities"].append(comtable[v.index])
plot_full(g, "full.svg")

In [ ]:
comtable=LouvainAlogrithm(g)

In [ ]:
for v in g.vs:
    v["communities"] = []
    v["communities"].append(comtable[v.index])
plot_full(g, "full.svg")

In [ ]:
print(comtable)

In [ ]:
g = read_nba_full("../../Data/Basketball/Players.csv",
                  "../../Data/Basketball/Edges.csv",
                  True, True)

In [ ]:
all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]

In [ ]:
def plotsubgraph(g,vertices):
    subg = g.subgraph(vertices)
 
    comtable=LouvainAlogrithm(subg)
    for v in subg.vs:
        v["communities"] = []
        v["communities"].append(comtable[v.index])
    plot_full(subg, "full.svg")
    i=0
    for v in subg.vs:
        v["community"] = comtable[i]
        i+=1
    print(GetModularity(subg))

In [ ]:
plotsubgraph(g,all11_20)

In [ ]:
plotsubgraph(g,all15_20)

In [ ]:
plotsubgraph(g,all19_20)

In [ ]:
q = g.community_multilevel()
p=g.modularity(q)

In [ ]:
def GetOverlapPercent(indices1, indices2):
    common = indices1 and indices2
    
def AssignCommunities(originaltable, currenttable):
    newtable = [0] * len(originaltable)
    for x in range(len(currenttable)):
        indices = GetIndices(currenttable)
        if len(indices)==0:
            continue
        for x in range(len(originaltable)):
            indices2 = GetIndices(currenttable)
            if len(indices2)==0:
                continue